## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
data_to_load = os.path.join("..", "Weather_Database", "WeatherPy_database.csv")
city_data_df = pd.read_csv(data_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,2021-10-11 00:07:20,-21.2078,-159.7750,78.85,73,100,9.22,overcast clouds
1,1,Poum,NC,2021-10-11 00:08:42,-20.2333,164.0167,75.42,65,11,18.66,few clouds
2,2,Yellowknife,CA,2021-10-11 00:04:32,62.4560,-114.3525,36.41,100,100,1.99,light rain
3,3,Hit,IQ,2021-10-11 00:08:42,33.6416,42.8251,76.91,20,0,8.50,clear sky
4,4,Upernavik,GL,2021-10-11 00:08:43,72.7868,-56.1549,33.33,97,100,10.87,light snow


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 95


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()

In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                476
City                   476
Country                476
Date                   476
Lat                    476
Lng                    476
Max Temp               476
Humidity               476
Cloudiness             476
Wind Speed             476
Current Description    476
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,
1,Poum,NC,75.42,few clouds,-20.2333,164.0167,
3,Hit,IQ,76.91,clear sky,33.6416,42.8251,
5,Moree,AU,64.42,overcast clouds,-29.4667,149.8500,
7,Karratha,AU,78.89,clear sky,-20.7377,116.8463,
9,Naze,JP,77.88,broken clouds,28.3667,129.4833,
10,Rio Grande,BR,63.95,overcast clouds,-32.0350,-52.0986,
11,Bilma,NE,81.88,scattered clouds,18.6853,12.9164,
14,Skibbereen,IE,51.60,scattered clouds,51.5500,-9.2667,
15,Puerto Ayora,EC,69.55,broken clouds,-0.7393,-90.3518,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,Paradise Inn
1,Poum,NC,75.42,few clouds,-20.2333,164.0167,Gîte kajeon
3,Hit,IQ,76.91,clear sky,33.6416,42.8251,صالون زيرو
5,Moree,AU,64.42,overcast clouds,-29.4667,149.8500,Molika Springs Motel
7,Karratha,AU,78.89,clear sky,-20.7377,116.8463,ibis Styles Karratha
9,Naze,JP,77.88,broken clouds,28.3667,129.4833,オーベルジュ・アガルイ
10,Rio Grande,BR,63.95,overcast clouds,-32.0350,-52.0986,Hotel Atlântico Rio Grande
11,Bilma,NE,81.88,scattered clouds,18.6853,12.9164,Expeditions Ténére voyages
14,Skibbereen,IE,51.60,scattered clouds,51.5500,-9.2667,The West Cork Hotel
15,Puerto Ayora,EC,69.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))